In [1]:
from idaes import dmf
from idaes.dmf.resource import TidyUnitData
from idaes.core.util.misc import extract_data

In [2]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.network import Arc, SequentialDecomposition
from idaes.core import FlowsheetBlock
from idaes.unit_models import (PressureChanger,
                               Mixer,
                               Separator as Splitter,
                               Heater,
                               StoichiometricReactor)

In [3]:
import hda_ideal_VLE_dmf as thermo_props

In [4]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})

In [5]:
m.fs.thermo_params = thermo_props.HDAParameterBlock()

In [7]:
from idaes.dmf import magics

In [8]:
%dmf ls

2019-08-12 22:59:17,452 [ERROR] idaes.dmf.magics: Error with magic command: Unrecognized command `ls`
